## Pipeline de preprocesamiento

En esta práctica vamos a trabajar en un pipeline de preprocesamiento. <br />
Generar pipelines que incluyan tanto las etapas de preprocesamiento como predicción tiene dos grandes ventajas: <br/>
    
1. Mejorar la implementación: Organizar el procesamiento para que sean fáciles de reproducir tanto las etapas de entrenamiento como las de predicción sobre nuevos datos.
2. Aplicar validación cruzada sobre parametrizaciones tanto del proprocesamiento como del algoritmo predictivo.
     
Para implementar clases propias en el contexto de un Pipeline es importante distinguir qué funcionalidad corresponde a cada etapa y debe implementarse en cada método. 

* Los hiperparámetros se setean como atributos del objeto sobre-escribiendo el método __init__

* Todo lo que se debe calcular sobre datos de entrenamiento, se ejecuta en método fit(). Los parámetros calculados a partir de datos de entrenamiento se guardan como propiedades del objeto.

* Todo lo que se debe aplicar a datos tanto de entrenamiento como de test, se ejecuta en el método transform(). Esta funcionalidad puede aprovechar los parámetros calculados en el fit() para ejecutar distintas transformaciones entre los datos. 


Recordemos el orden de ejecución de los pipelines de sklearn. 


<img src='pipeline-diagram.png'> </img>

## 1- Preprocesamiento con datos de desnutrición infantil

Este dataset proviene de un <a href='https://www.kaggle.com/c/eci2017-1'> concurso de Kaggle organizado por la ECI </a>. 

El dataset consiste de datos de consultas médicas anonimizadas de niños en varias regiones del país.  
La idea de la competencia es construir un modelo que permita alertar sobre el problema de desnutrición antes de que se produzca. 
La variable de respuesta, "decae", se construye observando que alguna de las variables clave de crecimiento (índice de masa corporal, altura o peso) cae a un desvío estándar por debajo de la media cuando antes no lo estaba. En las consultas donde se observa esta situación la variable decae toma el valor de 1. 



In [1]:
import pandas as pd
data = pd.read_csv('train_all_feats.csv')
data = data.drop('Unnamed: 0',axis=1)
data.head()

,BMIZ,HAZ,WAZ,individuo,bmi,departamento_indec_id,departamento_lat,departamento_long,fecha_control,fecha_nacimiento,...,zona_rural,decae,edad_dias,año_control,mes_control,dia_semana_control,edad_meses,dias_resto_mes,PCA1,PCA2
0,2.174022,-1.033244,0.950707,26316,19.852262,882.0,-34.096238,-59.028627,2013-09-20,2013-07-15,...,N,0.0,67.0,2013,9,4,2.0,65.0,-8.442610,-3.114742
1,2.997723,-1.302271,1.441404,26316,21.832807,882.0,-34.096238,-59.028627,2013-10-17,2013-07-15,...,N,0.0,94.0,2013,10,3,3.0,91.0,-6.009246,-5.326684
2,2.327996,-0.549524,1.472959,26316,21.003991,882.0,-34.096238,-59.028627,2014-03-07,2013-07-15,...,N,1.0,235.0,2014,3,4,7.0,228.0,4.720164,-5.319465
3,-0.532866,-2.195611,-1.651844,21124,16.158818,274.0,-34.794349,-58.264684,2013-10-16,2013-07-16,...,N,0.0,92.0,2013,10,2,3.0,89.0,-8.297273,2.739630
4,-0.522876,-0.507069,-0.683329,21124,16.568047,274.0,-34.794349,-58.264684,2013-12-18,2013-07-16,...,N,0.0,155.0,2013,12,2,5.0,150.0,-0.345926,0.842113


#### 1-1. Reorganización del conjunto de entrenamiento

La competencia propone desarrollar un modelo predictivo que indentifique la posibilidad de decaer en la cuarta consulta. 
Por eso en primer lugar es necesario reorganizar los datos de entrenamiento.

* Cada individuo debe ser una fila
* Cada consulta pasada debe estar modelada en varias columnas, a través de lags. 
* Los individuos que en lugar de 4 tienen sólo 3 consultas son los que hay que predecir en la competencia

In [2]:
data.columns

Index(['BMIZ', 'HAZ', 'WAZ', 'individuo', 'bmi', 'departamento_indec_id',
       'departamento_lat', 'departamento_long', 'fecha_control',
       'fecha_nacimiento', 'fecha_proximo_control', 'genero',
       'nombre_provincia', 'nombre_region', 'perimetro_encefalico', 'peso',
       'provincia_indec_id', 'talla', 'var_BMIZ', 'var_HAZ', 'var_WAZ',
       'zona_rural', 'decae', 'edad_dias', 'año_control', 'mes_control',
       'dia_semana_control', 'edad_meses', 'dias_resto_mes', 'PCA1', 'PCA2'],
      dtype='object')

In [3]:
# Generamos una nueva columna que indique la cantidad de controles que tuvo el individuo en total 
data['n_controles'] = 0
data['n_controles'] = data.groupby(['individuo'])['n_controles'].transform('count')
# Nos quedamos únicamente con los individuos que tienen 4 controles
df = data[data['n_controles']==4].copy()
df.head()

,BMIZ,HAZ,WAZ,individuo,bmi,departamento_indec_id,departamento_lat,departamento_long,fecha_control,fecha_nacimiento,...,decae,edad_dias,año_control,mes_control,dia_semana_control,edad_meses,dias_resto_mes,PCA1,PCA2,n_controles
3,-0.532866,-2.195611,-1.651844,21124,16.158818,274.0,-34.794349,-58.264684,2013-10-16,2013-07-16,...,0.0,92.0,2013,10,2,3.0,89.0,-8.297273,2.739630,4
4,-0.522876,-0.507069,-0.683329,21124,16.568047,274.0,-34.794349,-58.264684,2013-12-18,2013-07-16,...,0.0,155.0,2013,12,2,5.0,150.0,-0.345926,0.842113,4
5,-0.375011,-0.141067,-0.369380,21124,16.803193,274.0,-34.794349,-58.264684,2014-02-17,2013-07-16,...,0.0,216.0,2014,2,0,7.0,209.0,4.204053,0.244068,4
6,-0.690880,-0.282108,-0.683334,21124,16.326531,274.0,-34.794349,-58.264684,2014-03-17,2013-07-16,...,0.0,244.0,2014,3,0,8.0,236.0,5.547571,1.086780,4
10,0.189790,0.292899,0.304618,21134,17.169615,357.0,-38.024711,-57.556547,2013-10-24,2013-07-25,...,0.0,91.0,2013,10,3,3.0,88.0,-3.890028,-0.570522,4


In [4]:
# Calculamos para cada registro, qué número de control DEL individuo representa y lo guardamos en la columna nro_control
df['nro_control'] = df.groupby(['individuo'])['fecha_control'].rank(ascending=True)
# Luego ordenamos el dataframe por individuo y fecha de control.
df = df.sort_values(['individuo','fecha_control'])

In [5]:
df[['BMIZ', 'HAZ', 'WAZ', 'individuo','fecha_control',
       'fecha_nacimiento', 'fecha_proximo_control',
       'n_controles', 'nro_control']].head(12)

,BMIZ,HAZ,WAZ,individuo,fecha_control,fecha_nacimiento,fecha_proximo_control,n_controles,nro_control
38903,0.087923,0.413162,0.301353,22,2014-07-04,2014-05-22,2014-08-04,4,1.0
38904,0.758371,-0.150787,0.474523,22,2014-08-04,2014-05-22,2014-08-28,4,2.0
38905,-0.036034,0.783899,0.411194,22,2014-08-28,2014-05-22,2014-10-06,4,3.0
38906,1.189397,0.867424,1.366334,22,2014-10-06,2014-05-22,2014-11-17,4,4.0
5322,-0.000185,0.534095,0.278458,37,2014-02-18,2014-01-24,2014-03-25,4,1.0
5323,-1.179932,1.836000,0.157728,37,2014-03-25,2014-01-24,2014-04-29,4,2.0
5324,-0.956759,1.584035,0.183990,37,2014-04-29,2014-01-24,2014-05-27,4,3.0
5325,-0.519260,1.932113,0.700032,37,2014-05-27,2014-01-24,2014-07-11,4,4.0
40446,1.074068,-2.249381,-0.513327,68,2014-05-30,2014-03-13,2014-07-07,4,1.0
40447,0.730170,-0.783378,0.097087,68,2014-07-07,2014-03-13,2014-08-06,4,2.0


A continuación creamos lags para las variables indicadoras, para representar la evolución del individuo en los controles anteriores. 
Para esto utilizamos la función shift de pandas y creamos nuevas columnas. 

In [6]:
df['BMIZ_l1'] = df.groupby(['individuo'])['BMIZ'].shift(1)
df['HAZ_l1'] = df.groupby(['individuo'])['HAZ'].shift(1)
df['WAZ_l1'] = df.groupby(['individuo'])['WAZ'].shift(1)
df['BMIZ_l2'] = df.groupby(['individuo'])['BMIZ'].shift(2)
df['HAZ_l2'] = df.groupby(['individuo'])['HAZ'].shift(2)
df['WAZ_l2'] = df.groupby(['individuo'])['WAZ'].shift(2)
df['BMIZ_l3'] = df.groupby(['individuo'])['BMIZ'].shift(3)
df['HAZ_l3'] = df.groupby(['individuo'])['HAZ'].shift(3)
df['WAZ_l3'] = df.groupby(['individuo'])['WAZ'].shift(3)

Por último para entrenar el modelo nos quedamos con las cuartas consultas de cada individuo:

In [7]:
df = df[df['nro_control']==4].copy()

#### 1-1. Completar datos faltantes

Veamos qué columnas presentan datos faltantes:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6132 entries, 38906 to 30417
Data columns (total 42 columns):
BMIZ                     6132 non-null float64
HAZ                      6132 non-null float64
WAZ                      6132 non-null float64
individuo                6132 non-null int64
bmi                      6132 non-null float64
departamento_indec_id    5585 non-null float64
departamento_lat         5585 non-null float64
departamento_long        5585 non-null float64
fecha_control            6132 non-null object
fecha_nacimiento         6132 non-null object
fecha_proximo_control    6132 non-null object
genero                   6132 non-null object
nombre_provincia         5585 non-null object
nombre_region            5585 non-null object
perimetro_encefalico     6132 non-null float64
peso                     6132 non-null float64
provincia_indec_id       5585 non-null float64
talla                    6132 non-null float64
var_BMIZ                 6132 non-null float64
var

Los únicos datos faltantes se relacionan con la ubicación geográfica. 

#### 1-2. Pipeline para variables categóricas

Para trabajar con las variables categóricas, algunas de las cuales incluyen datos faltantes, vamos a generar pipelines que incluyan los siguientes pasos 

1. Seleccionar el ítem (columna)categórica sobre la cual trabajar
2. Llenar na con una categoría específica NA (tratamos que el algoritmo aprenda las características de un dato faltante)
3. Colapsar categorías con muy pocas ocurrencias
4. Crear Dummies

Para esto vamos a construir clases propias que hereden de BaseEstimator y TransformerMixin para poder utilizarlas en un pipeline más general. 

Es importante que los métodos fit() estén preparados para recibir el parámetro y aunque no hagan nada con él, ya que el pipeline generalmente se entrena sobre los vectores X_train e y_train. 

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
# Selecciona una lista de columnas para procesar.
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X[self.key]
        return X

In [10]:
# Esta clase asigna el string 'NA' a cualquier valor faltante que se encuentra en la categoría.

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
class CategoricalImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.apply(lambda val: val.astype(str))
        X = X.fillna('NA')
        return X

In [11]:
# Esta clase colapsa todas aquellas categorías que aparecen menos de cierta cantidad de veces.
# Si un valor es demasiado infrecuente es difícil que un modelo predictivo pueda usarlo para aprender.

class CategoricalReduction(BaseEstimator, TransformerMixin):
    def __init__(self, min_apariciones):
        self.min_apariciones = min_apariciones

    def fit(self, X, y=None):
        self.items_raros = (X.iloc[:,0].value_counts()[X.iloc[:,0].value_counts() < self.min_apariciones]).index.values
        return self

    def transform(self, X):
        colnames = X.columns
        X.loc[X.iloc[:,0].isin(list(self.items_raros)), colnames[0]] = 'Others'
        return X

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
class LabelBinarizer_new(TransformerMixin, BaseEstimator):
    def fit(self, X, y = 0):
        encoder = LabelBinarizer();
        encoder.fit(X)
        self.encoder = encoder
        return self
    def transform(self, X):
        X = self.encoder.transform(X)
        return X;   

In [13]:
vars_categoricas = ['departamento_indec_id','genero', 'nombre_provincia', 'nombre_region','zona_rural']

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
# Construimos una lista de tuplas para crear una unión de todos los pipelines.
steps = []
for var in vars_categoricas:
        steps.append(('cat_' + var,
            Pipeline([
                ('itemSel',ItemSelector([var])),
                ('catImpute',CategoricalImputer()),
                ('catReducr',CategoricalReduction(4)),
                ('le',LabelBinarizer_new())])
        ))      

In [15]:
from sklearn.preprocessing import Imputer
# Seleccionamos las variables numéricas
pipe_numerica = Pipeline([
                    ('itemSelNum',
                      ItemSelector(['BMIZ', 'HAZ', 'WAZ', 'individuo', 'bmi', 
                    'departamento_lat', 'departamento_long',
                    'perimetro_encefalico', 'peso', 'provincia_indec_id', 'talla', 'edad_dias', 
                    'año_control', 'mes_control','dia_semana_control', 'edad_meses', 'dias_resto_mes', 'PCA1', 'PCA2',
                    'BMIZ_l1', 'HAZ_l1', 'WAZ_l1', 'BMIZ_l2','HAZ_l2', 'WAZ_l2', 'BMIZ_l3', 'HAZ_l3', 'WAZ_l3'])),
                    ('imputer',Imputer())
                            ])

In [16]:
steps.append(('pipe_numerica',pipe_numerica))

In [17]:
from sklearn.pipeline import FeatureUnion

preprocesamiento_pipe = FeatureUnion(steps)

In [18]:
preprocesamiento_pipe.fit_transform(df[['BMIZ', 'HAZ', 'WAZ', 'individuo', 'bmi', 'departamento_indec_id',
'departamento_lat', 'departamento_long', 'genero', 'nombre_provincia', 'nombre_region', 
'perimetro_encefalico', 'peso', 'provincia_indec_id', 'talla','zona_rural', 'edad_dias', 
'año_control', 'mes_control','dia_semana_control', 'edad_meses', 'dias_resto_mes', 'PCA1', 'PCA2',
'BMIZ_l1', 'HAZ_l1', 'WAZ_l1', 'BMIZ_l2','HAZ_l2', 'WAZ_l2', 'BMIZ_l3', 'HAZ_l3', 'WAZ_l3']],df['decae'])

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          8.79226541e-02,   4.13162174e-01,   3.01352785e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.84747245e-04,   5.34095314e-01,   2.78457950e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.07406779e+00,  -2.24938101e+00,  -5.13326502e-01],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          3.96510652e-01,  -3.30753407e-01,   1.10701815e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.33607128e-01,  -1.58603843e+00,  -3.34051715e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          4.13153117e-01,   1.02072454e-01,   3.67236594e-01]])